In [1]:
from functools import partial

import numpy as np

import torch
import torch.nn as nn
import torch_points_kernels as tp
from torch.utils.data import DataLoader
from torch_points3d.core.common_modules import FastBatchNorm1d
from torch_points3d.modules.KPConv.kernels import KPConvLayer
from torch_geometric.nn import voxel_grid


from lib.pointops2.functions import pointops

from util.data_util import collate_fn_limit
from util.s3dis import S3DIS

In [3]:
dev = 'cuda:0'
cuda_idx = 0
device = torch.device(dev)
torch.cuda.set_device(device)

## Data Setup

In [4]:
data_root = '/home/Pointnet_Pointnet2_pytorch/data/stanford_indoor3d'
train_transform = None
train_data = S3DIS(split='train', data_root=data_root, test_area=5, voxel_size=0.04, voxel_max=80000,
                   transform=train_transform)

Totally 204 samples in train set.


In [5]:
batch_size = 2
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, pin_memory=False, 
                          drop_last=True, collate_fn=partial(collate_fn_limit, max_batch_points=200000, logger=None)
                        )

## KPConv

In [6]:
class KPConvSimpleBlock(nn.Module):
    def __init__(self, in_channels, out_channels, prev_grid_size, sigma=1.0, negative_slope=0.2, bn_momentum=0.02):
        super().__init__()
        self.kpconv = KPConvLayer(in_channels, out_channels, point_influence=prev_grid_size * sigma, add_one=False)
        self.bn = FastBatchNorm1d(out_channels, momentum=bn_momentum)
        self.activation = nn.LeakyReLU(negative_slope=negative_slope)

    def forward(self, feats, xyz, batch, neighbor_idx):
        # feats: [N, C]
        # xyz: [N, 3]
        # batch: [N,]
        # neighbor_idx: [N, M]
        feats = self.kpconv(xyz, xyz, neighbor_idx, feats)
        feats = self.activation(self.bn(feats))
        return feats


class KPConvResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, prev_grid_size, sigma=1.0, negative_slope=0.2, bn_momentum=0.02):
        super().__init__()
        d_2 = out_channels // 4
        activation = nn.LeakyReLU(negative_slope=negative_slope)
        self.unary_1 = torch.nn.Sequential(nn.Linear(in_channels, d_2, bias=False), FastBatchNorm1d(d_2, momentum=bn_momentum), activation)
        self.unary_2 = torch.nn.Sequential(nn.Linear(d_2, out_channels, bias=False), FastBatchNorm1d(out_channels, momentum=bn_momentum), activation)
        self.kpconv = KPConvLayer(d_2, d_2, point_influence=prev_grid_size * sigma, add_one=False)
        self.bn = FastBatchNorm1d(out_channels, momentum=bn_momentum)
        self.activation = activation

        if in_channels != out_channels:
            self.shortcut_op = torch.nn.Sequential(
                nn.Linear(in_channels, out_channels, bias=False), FastBatchNorm1d(out_channels, momentum=bn_momentum)
            )
        else:
            self.shortcut_op = nn.Identity()

    def forward(self, feats, xyz, batch, neighbor_idx):
        # feats: [N, C]
        # xyz: [N, 3]
        # batch: [N,]
        # neighbor_idx: [N, M]
        
        shortcut = feats
        feats = self.unary_1(feats)
        feats = self.kpconv(xyz, xyz, neighbor_idx, feats)
        feats = self.unary_2(feats)
        shortcut = self.shortcut_op(shortcut)
        feats += shortcut
        return feats

## HOG

In [20]:
def compute_hog(xyz, neighbor_idx):
    '''
    Histogram of Oriented Gradients in Non-Overlapping Cubic Windows
    
    1. Query neighbors of all points
    2. Reduce each neighborhood by applying SVD
    3. Values obtained are s (magnitude) and v[0] (gradient)
    4. get angles from gradients & convert angles to unsigned
    5. initialize histogram for each cubic window as 2D (zenith & azimuth) array of 9 cells (20 degrees each)
    6. each window contributes its gradient to the respective cell of the histogram as per its angles
    8. normalize gradients of each window by
        (option 1) considering that window only
        (option 2) considering 3 adjacent windows
    9. flatten the histogram and return it (n, 18)
    '''
    device = xyz.get_device()
    # 1. Query neighbors of all points & get displacement to neighbors
    disp = pointops.subtraction(xyz, xyz, neighbor_idx.int())
    # 0 distance to itself
    disp[neighbor_idx == -1] = 0
    # 2. Reduce each neighborhood by applying SVD
    _, s, v = np.linalg.svd(disp.cpu().numpy(), full_matrices = False)
    # 3. Values obtained are s (magnitude) and v[0] (gradient)
    # get the first element (largest variance)
    magnitudes = s[:, 0].unsqueeze(-1).cuda(device) # N x 3 -> N x 1
    gradients = v[:, :, 0].cuda(device)  #  N x 3 x 3 -> N x 3
    # 4. get angles from gradients & convert angles to unsigned
    zenith = torch.acos(gradients[:, 2]) * 180 / np.pi
    # add 1e-9 for safe division
    azimuth = torch.atan(torch.div(gradients[:, 1], gradients[:, 0] + 1e-9)) * 180 / np.pi
    

## Run!

In [7]:
emb = nn.ModuleList([
    KPConvSimpleBlock(6, 48, 0.04, sigma=1.0),
    KPConvResBlock(48, 48, 0.04, sigma=1.0)
]).cuda()


In [8]:
max_num_neighbors = 40
# (n, 3), (n, c), (n), (b)
coord, feat, target, offset = iter(train_loader).next()
offset_ = offset.clone()
offset_[1:] = offset_[1:] - offset_[:-1]
print('offset=', offset)
print('offset_=', offset_)
batch = torch.cat([torch.tensor([ii]*o) for ii, o in enumerate(offset_)], 0).long()

sigma = 1.0
radius = 2.5 * 0.04 * sigma
neighbor_idx = tp.ball_query(radius, max_num_neighbors, coord, coord, mode="partial_dense", batch_x=batch, batch_y=batch)[0]

coord, feat, target, offset, batch, neighbor_idx = coord.cuda(non_blocking=True), feat.cuda(non_blocking=True), \
                                    target.cuda(non_blocking=True), offset.cuda(non_blocking=True), \
                                    batch.cuda(non_blocking=True), neighbor_idx.cuda(non_blocking=True)
assert batch.shape[0] == feat.shape[0]
feat = torch.cat([feat, coord], 1)

offset= tensor([ 43340,  54990,  77996,  88110, 111057, 185809], dtype=torch.int32)
offset_= tensor([43340, 11650, 23006, 10114, 22947, 74752], dtype=torch.int32)


In [27]:
def grid_sample(pos, batch, size, start, return_p2v=True):
    # pos: float [N, 3]
    # batch: long [N]
    # size: float [3, ]
    # start: float [3, ] / None

    cluster = voxel_grid(pos, batch, size, start=start) #[N, ]

    if return_p2v == False:
        unique, cluster = torch.unique(cluster, sorted=True, return_inverse=True)
        return cluster

    unique, cluster, counts = torch.unique(cluster, sorted=True, return_inverse=True, return_counts=True)

    # obtain p2v_map
    n = unique.shape[0]
    k = counts.max().item()
    p2v_map = cluster.new_zeros(n, k) #[n, k]
    mask = torch.arange(k).cuda().unsqueeze(0) < counts.unsqueeze(-1) #[n, k]
    p2v_map[mask] = torch.argsort(cluster)

    return cluster, p2v_map, counts

In [33]:
v2p_map, p2v_map, counts = grid_sample(coord, batch, 4, start=None)

In [34]:
print(v2p_map.shape)
print(p2v_map.shape)
print(counts.shape)

torch.Size([185809])
torch.Size([10, 43340])
torch.Size([10])


In [35]:
counts

tensor([43340, 11650, 23006, 10114, 20055,  2892, 28006, 16850, 22413,  7483],
       device='cuda:1')